<a href="https://colab.research.google.com/github/sftSalman/DCGAN_for-generating-satelite-images/blob/main/DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import torch 
import torch.nn as nn

In [25]:
class Discriminator(nn.Module):

  def __init__(self, channels_img, features_d):
    super(Discriminator,self).__init__()

    self.disc = nn.Sequential(
        
        nn.Conv2d(channels_img,features_d,kernel_size=4,stride=2,padding=1),
        nn.LeakyReLU(0.2),
        self.block(features_d, features_d*2 , 4, 2,1),
        self.block(features_d*2, features_d*4 , 4, 2,1),
        self.block(features_d*4, features_d*8 , 4, 2,1),

        nn.Conv2d(features_d * 8, 1,kernel_size=4,stride=2,padding=0),
        nn.Sigmoid(),

        
   


    )


  def block(self,in_channels,out_channels,kernel_size,stride,padding):

    return nn.Sequential(
        nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size,
            stride,
            padding,
            bias = False
          ),
       # nn.BatchNorm1d2d(out_channels),
        nn.LeakyReLU(0.2)

      )

  def forward(self,x):
    return self.disc(x)

In [26]:
class Generator(nn.Module):
  def __init__(self,channels_noise,channels_img,features_g):
    super(Generator,self).__init__()

    self.net = nn.Sequential(
        self.block(channels_noise,features_g*16,4,1,0),
        self.block(features_g * 16, features_g * 8, 4, 2, 1),
        self.block(features_g * 8, features_g * 4, 4, 2, 1),
        self.block(features_g * 4, features_g * 2, 4, 2, 1),

        nn.ConvTranspose2d(features_g*2, channels_img,kernel_size=4,stride=2,padding=1),
        nn.Tanh(),
        
    )
  
  def block(self, in_channels,out_channels,kernel_size,stride,padding):
    return nn.Sequential(
        nn.ConvTranspose2d(
            in_channels,
            out_channels,
            kernel_size,
            stride,
            padding,
            bias = False
        ),
        nn.ReLU()

    )
  
  def forward(self,x):
    return self.net(x)



In [40]:
def initialize_weights(models):
  for m in models.modules():
    if isinstance(m,(nn.Conv2d,nn.ConvTranspose2d,nn.BatchNorm2d)):
      nn.init.normal_(m.weight.data, 0.0, 0.02)

In [28]:
def test():
    N, in_channels, H, W = 8, 3, 64, 64
    noise_dim = 100
    x = torch.randn((N, in_channels, H, W))
    disc = Discriminator(in_channels, 8)
    assert disc(x).shape == (N, 1, 1, 1), "Discriminator test failed"
    gen = Generator(noise_dim, in_channels, 8)
    z = torch.randn((N, noise_dim, 1, 1))
    assert gen(z).shape == (N, in_channels, H, W), "Generator test failed"
    print("Success, tests passed!")

In [44]:
test()

Success, tests passed!


In [29]:
import torch.optim as optim 
import torchvision
import torchvision.datasets as datasets
#from torchvision.transforms import transfroms 
import torchvision.transforms as transfoms
#from torch.utils.data import Dataloader 
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter


In [30]:
LEARNING_RAT = 2e-4 
BATCH_SIZE = 128
IMAGE_SIZE = 64
CHANNELS_IMG = 1 
NOISE_DIM = 100
NUM_EPOCHS = 5 
FEATURES_DISC = 64 
FEATURES_GEN = 64

In [8]:
pip install torch


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:

import torch
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [33]:
from torchvision import transforms

transforms = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.5 for _ in range(CHANNELS_IMG)], [0.5 for _ in range(CHANNELS_IMG)])
])


In [ ]:
# dataset = datasets.MNIST(
#     root="dataset/", train=True, transform=transforms, download=True
# )

In [37]:
dataloader = DataLoader(datasets,batch_size=BATCH_SIZE,shuffle=True)
gen = Generator(NOISE_DIM,CHANNELS_IMG,FEATURES_GEN).to(device)
disc = Discriminator(CHANNELS_IMG,FEATURES_DISC).to(device)

In [41]:
initialize_weights(gen)
initialize_weights(disc)

In [42]:
opt_gen = optim.Adam(gen.parameters(),lr=LEARNING_RAT,betas=(0.5,0.999))
opt_gen = optim.Adam(disc.parameters(),lr=LEARNING_RAT,betas=(0.5,0.999))
criterian = nn.BCELoss()

In [43]:
gen.train()
disc.train()

Discriminator(
  (disc): Sequential(
    (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2)
    (2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): LeakyReLU(negative_slope=0.2)
    )
    (3): Sequential(
      (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): LeakyReLU(negative_slope=0.2)
    )
    (4): Sequential(
      (0): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): LeakyReLU(negative_slope=0.2)
    )
    (5): Conv2d(512, 1, kernel_size=(4, 4), stride=(2, 2))
    (6): Sigmoid()
  )
)

In [ ]:
for epoch in range(NUM_EPOCHS):
  for batch_idx,(real,_) in enumerate(dataloader):
    real = real.to(device)
    noise = torch.randn(BATCH_SIZE,NOISE_DIM,1,1).to(device)
    fake = gen(noise)
    disc_real = criterian(d)